In [1]:
!pip install scikit-surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-win_amd64.whl size=759505 sha256=79df0f91f341e291c750fa3e82655a61a35148ed6c7b7339fadf6a0528f9c4e8
  Stored in directory: c:\users\mylaptop\appdata\local\pip\cache\wheels\76\44\74\b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


You should consider upgrading via the 'c:\users\mylaptop\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from surprise import SVD, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

#### 데이터셋 불러오기

In [3]:
ratings = pd.read_csv("./data/ratings.csv")
print(ratings)

        userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]


In [8]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(df=ratings[['userId', 'movieId', 'rating']], reader=reader)
print(data)

In [10]:
train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=23)

# 원본행렬에 가깝도록 trainset을 사용해서 행렬을 분해
# 분해된 행렬을 연산하면 원본 행렬을 만들어 낼 수 있도록
algo = SVD(n_factors=50, n_epochs=20)
algo.fit(trainset=train)

In [15]:
# 잘 분해가 되었는지 정확도 확인
pred = algo.test(testset=test)
# print(pred[0:3])
accuracy.rmse(predictions=pred)

RMSE: 0.8860


0.8860403531101189

In [16]:
# 분해된 행렬을 연산을 해서 예측값을 도출해냄
pred = algo.predict('10', '200') # userId, movieId
print(pred) # 지금 데이터상으로는 아무런 평점 데이터가 없다.

user: 10         item: 200        r_ui = None   est = 3.54   {'was_impossible': False}


In [22]:
# 10번 유저에 대해서 모든 영화에 부여할 평점을 예측
preds = []
for i in ratings['movieId'].unique(): # 영화평점은 중복된게 있으므로
    pred = algo.predict('10', i)
    preds.append((pred.est, pred.iid))

print(preds)

[(3.359118862254692, 31), (3.745095596843024, 1029), (3.670134297735522, 1061), (3.4068975213530144, 1129), (4.185119979579551, 1172), (3.899220390951424, 1263), (3.9924692075455184, 1287), (4.118261300511852, 1293), (3.4453986539093444, 1339), (3.826958954499667, 1343), (3.1772937966534127, 1371), (3.301337245231667, 1405), (4.103471928533363, 1953), (3.6355993437313536, 2105), (3.641295499571871, 2150), (3.2990498976706633, 2193), (3.3374238525934548, 2294), (3.546391433486629, 2455), (3.6250800268283343, 2968), (4.045485207372025, 3671), (3.4628491956832366, 10), (3.9058435257545443, 17), (3.6061795247920583, 39), (3.9705877861061, 47), (4.370374600900331, 50), (3.6108527490214515, 52), (3.7232126109508714, 62), (3.895873715981649, 110), (3.384831978458258, 144), (3.9339899960455584, 150), (2.8992847140348967, 153), (3.8588642103102058, 161), (3.4507959036743543, 165), (3.2137884622895903, 168), (3.120657244840634, 185), (2.885347386882321, 186), (2.8546419133116627, 208), (3.880297

In [23]:
# 그 평점을 
preds.sort(reverse=True) # 평점이 높은 순서대로 재정렬
print(preds)

[(4.479457953626824, 318), (4.477020599386526, 858), (4.476969664563979, 969), (4.428136862191981, 1221), (4.381409153179034, 1254), (4.370374600900331, 50), (4.370335935678907, 1945), (4.368121413935237, 306), (4.350659635731727, 1228), (4.347342160420194, 2064), (4.340029033814393, 926), (4.335338578748208, 1252), (4.3304175060012, 3462), (4.328615770164309, 913), (4.325768565451465, 1217), (4.324466317629011, 905), (4.315893457835029, 7502), (4.3074209019507235, 527), (4.291457691890919, 908), (4.288405948403662, 2019), (4.284351283544571, 1247), (4.281920410858115, 1136), (4.281361225164827, 1203), (4.275349858234391, 912), (4.274181399964002, 2542), (4.272104983797428, 1193), (4.270063119245936, 2318), (4.2684679028908326, 1299), (4.2676153339817455, 2692), (4.26399545773277, 922), (4.2628618458180405, 899), (4.257103042098458, 6016), (4.2551579028005495, 608), (4.254400115022218, 1213), (4.2535594766915334, 904), (4.251941483264575, 1212), (4.244523933884598, 1060), (4.2392189008

In [27]:
# 평점이 높은 318번 영화가 뭔지 찾아보자
movies = pd.read_csv("./data/movies.csv")
print(movies[movies['movieId']==318])

     movieId                             title       genres
284      318  Shawshank Redemption, The (1994)  Crime|Drama
